In [2]:
!pip install -qU xpander-sdk llama-index llama-index-llms-openai python-dotenv


In [1]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool
from os import environ
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = environ["OPENAI_API_KEY"]
XPANDER_API_KEY = environ["XPANDER_API_KEY"]
XPANDER_AGENT_ID = environ["XPANDER_AGENT_ID"]

In [2]:
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

def square(a: int) -> int:
    """Multiply a by itself and returns the result integer"""
    return a*a

square_tool = FunctionTool.from_defaults(fn=square)

In [3]:
# llm = OpenAI(model="gpt-3.5-turbo")
llm = OpenAI(model="gpt-4")
agent = ReActAgent.from_tools([multiply_tool, add_tool, square_tool], llm=llm, verbose=True)

In [4]:
response_gen = agent.stream_chat("What is 20+2*4? Calculate step by step")
response_gen.print_response_stream()

> Running step 1965c8da-b79d-4f29-9195-7a31889919ed. Step input: What is 20+2*4? Calculate step by step
Thought: The user wants to calculate the expression 20+2*4 step by step. According to the order of operations (BIDMAS/BODMAS), multiplication should be performed before addition. So, I will first multiply 2 and 4, then add the result to 20.
Action: multiply
Action Input: {'a': 2, 'b': 4}
Observation: 8
> Running step 43e9129b-4113-40dc-a3d1-634deba1cb7f. Step input: None
Thought: The multiplication of 2 and 4 is 8. Now, I will add this result to 20.
Action: add
Action Input: {'a': 20, 'b': 8}
Observation: 28
> Running step 3602cf7a-da62-46be-9a5e-03c870ba6058. Step input: None
 The result of the calculation 20+2*4 is 28. Here's the step by step calculation: First, we multiply 2 and 4 which gives 8. Then, we add this result to 20, which gives the final result 28.

In [ ]:
response_gen_3 = agent.stream_chat("What is the square of 14 + 12 squared? Calculate step by step")

> Running step 94b6da96-ac34-495c-be36-206923cbf2dd. Step input: What is the square of 14 + 12 squared? Calculate step by step
Thought: The user wants to calculate the square of 14 and add it to the square of 12. This involves two steps: squaring 14 and 12 separately, and then adding the results. I'll use the 'square' tool for the squaring operations and the 'add' tool for the addition.
Action: square
Action Input: {'a': 14}
Observation: 196
> Running step 4a1f8e32-3a5d-4de1-b9f1-716351e92aa8. Step input: None
Thought: The square of 14 is 196. Now, I need to calculate the square of 12.
Action: square
Action Input: {'a': 12}
Observation: 144
> Running step f4f7503a-4286-4b13-9ad8-6e2aa7cc7f3a. Step input: None
Thought: The square of 12 is 144. Now, I need to add 196 (the square of 14) and 144 (the square of 12).
Action: add
Action Input: {'a': 196, 'b': 144}
Observation: 340
> Running step 1c265b36-1cef-421d-87e8-74a7dbf71717. Step input: None


In [7]:
response_gen_3.print_response_stream()

 The square of 14 plus the square of 12 equals 340.

In [ ]:
response_gen_4 = agent.stream_chat("What is the square of 14 + the square of 12? Calculate step by step")
response_gen_4.print_response_stream()

> Running step 92c997e3-421f-4c44-8164-14bad8693a31. Step input: What is the square of 14 + the square of 12? Calculate step by step
Thought: The user is asking for the square of 14 and the square of 12. I need to use the square tool twice to get these values.
Action: square
Action Input: {'a': 14}
Observation: 196
> Running step 2ec200cc-28c5-42c3-a170-ca3e9b80fd93. Step input: None
Thought: I have the square of 14, which is 196. Now I need to calculate the square of 12.
Action: square
Action Input: {'a': 12}
Observation: 144
> Running step f7412580-5045-4092-9c82-2b48772a7979. Step input: None
Thought: I have the square of 12, which is 144. Now I need to add these two squares together.
Action: add
Action Input: {'a': 196, 'b': 144}
Observation: 340
> Running step c8c45af9-fb97-4876-8361-bd179f6fe0ca. Step input: None
 The square of 14 plus the square of 12 equals 340.

In [5]:
from openai import OpenAI
from xpander_sdk import XpanderClient, LLMProvider
# load the client
xpander_client = XpanderClient(api_key=XPANDER_API_KEY, organization_id="")

# get the agent
xpander_agent = xpander_client.agents.get(agent_id=XPANDER_AGENT_ID)
openai_client = OpenAI(api_key=OPENAI_API_KEY)

b'\x1b\x1b!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!\x1b\x1b\n'
b'\x1b\x1b!!                                                                                                                      !!\x1b\x1b\n'
b'\x1b\x1b!!  This software has not been tested with node v23.4.0.                                                                !!\x1b\x1b\n'
b'\x1b\x1b!!  Should you encounter odd runtime issues, please try using one of the supported release before filing a bug report.  !!\x1b\x1b\n'
b'\x1b\x1b!!                                                                                                                      !!\x1b\x1b\n'
b'\x1b\x1b!!  This software is currently running on node v23.4.0.                                                                 !!\x1b\x1b\n'
b'\x1b\x1b!!  As of the current release of this software, supported node releases are:                                            !!\x1b

In [6]:
local_tools = [
    {
        "type": "function",
        "function": {
            "name": "multiply",
            "description": "Multiply two integers and returns the result integer",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {
                        "type": "integer",
                        "description": "First integer to multiply"
                    },
                    "b": {
                        "type": "integer",
                        "description": "Second integer to multiply"
                    }
                },
                "required": ["a", "b"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "add",
            "description": "Add two integers and returns the result integer",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {
                        "type": "integer",
                        "description": "First integer to add"
                    },
                    "b": {
                        "type": "integer",
                        "description": "Second integer to add"
                    }
                },
                "required": ["a", "b"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "square",
            "description": "Multiply a by itself and returns the result integer",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {
                        "type": "integer",
                        "description": "Integer to square"
                    }
                },
                "required": ["a"]
            }
        }
    }
]


xpander_agent.add_local_tools(local_tools)
xpander_agent.get_tools()

[{'type': 'function',
  'function': {'name': 'xpLocal_multiply',
   'description': 'Multiply two integers and returns the result integer',
   'parameters': {'type': 'object',
    'properties': {'a': {'type': 'integer',
      'description': 'First integer to multiply'},
     'b': {'type': 'integer', 'description': 'Second integer to multiply'}},
    'required': ['a', 'b']}}},
 {'type': 'function',
  'function': {'name': 'xpLocal_add',
   'description': 'Add two integers and returns the result integer',
   'parameters': {'type': 'object',
    'properties': {'a': {'type': 'integer',
      'description': 'First integer to add'},
     'b': {'type': 'integer', 'description': 'Second integer to add'}},
    'required': ['a', 'b']}}},
 {'type': 'function',
  'function': {'name': 'xpLocal_square',
   'description': 'Multiply a by itself and returns the result integer',
   'parameters': {'type': 'object',
    'properties': {'a': {'type': 'integer',
      'description': 'Integer to square'}},
    

In [7]:

def handle_local_tool_calls(tool_call):
    """Handle local tool calls for calculator functions"""
    result = None
    params = tool_call.payload.get("bodyParams", {})
    
    if tool_call.name == "multiply":
        result = multiply(
            params.get("a"),
            params.get("b")
        )
    elif tool_call.name == "add":
        result = add(
            params.get("a"),
            params.get("b")
        )
    elif tool_call.name == "square":
        result = square(
            params.get("a")
        )
    
    return result

In [8]:
from xpander_sdk import ToolCallType
# create execution

xpander_agent.add_task("What is the square of 14 + the square of 12? Calculate step by step")

while not xpander_agent.is_finished():
    print("-"*100)
    response = openai_client.chat.completions.create(
                model= 'gpt-4o',
                messages=xpander_agent.memory.retrieve_messages(),
                tools=xpander_agent.get_tools(llm_provider=LLMProvider.OPEN_AI),
                tool_choice="auto",
                temperature=0.0
        )
    print(response.model_dump())
    # add messages directly from the LLM response
    xpander_agent.memory.add_messages(response.model_dump())
    
    # extract tool calls from the LLM response
    tool_calls = XpanderClient.extract_tool_calls(llm_response=response.model_dump(),llm_provider=LLMProvider.OPEN_AI)
    # run tools
    for tool_call in tool_calls:
        if tool_call.type == ToolCallType.LOCAL:
            result = handle_local_tool_calls(tool_call)
            xpander_agent.add_tool_result(tool_call=tool_call, result=result)
        else:
            xpander_agent.run_tool(tool_call=tool_call)
    
print(xpander_agent.retrieve_execution_result().result)

----------------------------------------------------------------------------------------------------
{'id': 'chatcmpl-AvbFHc96ZnDQslA8WQsZBpXPvMu4R', 'choices': [{'finish_reason': 'tool_calls', 'index': 0, 'logprobs': None, 'message': {'content': None, 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': [{'id': 'call_tsJFzhtxR9CaimqgkHHcZvhA', 'function': {'arguments': '{"a": 14}', 'name': 'xpLocal_square'}, 'type': 'function'}, {'id': 'call_I0191nT1orgXs1rEJUXfNVFb', 'function': {'arguments': '{"a": 12}', 'name': 'xpLocal_square'}, 'type': 'function'}]}}], 'created': 1738291231, 'model': 'gpt-4o-2024-08-06', 'object': 'chat.completion', 'service_tier': 'default', 'system_fingerprint': 'fp_50cad350e4', 'usage': {'completion_tokens': 49, 'prompt_tokens': 2327, 'total_tokens': 2376, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audi

TypeError: unsupported operand type(s) for *: 'NoneType' and 'NoneType'